<a href="https://colab.research.google.com/github/varunbhanot/NLP/blob/master/Sentiment_Analysis_With_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1 : Load the file

In [0]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [0]:
df = pd.read_csv("/content/smileannotationsfinal.csv" , names=['id','text','category'])
df.set_index('id', inplace=True)

In [14]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [0]:
df = df[df.category != 'nocode' ]
df = df[~df.category.str.contains('\|')]

In [17]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [0]:
labels_dict={}
for index,possible_label in enumerate(df.category.unique()):
  labels_dict[possible_label] = index

In [19]:
labels_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [21]:
df['label'] = df.category.replace(labels_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


# Task 2 : Train/Test Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_val , y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=17,
    stratify = df.label.values
)

In [0]:
df['data_type'] = ['not_set'] * df.shape[0]

In [0]:
df.loc[X_train,'data_type'] = 'train'
df.loc[X_val,'data_type'] = 'val'

In [29]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,train
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,train
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train


In [31]:
df.groupby(['category','label','data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

# Task 3 : Tokenizer

In [33]:
!pip install transformers

     |████████████████████████████████| 665kB 9.0MB/s 
     |████████████████████████████████| 3.8MB 52.5MB/s 
     |████████████████████████████████| 890kB 42.8MB/s 
     |████████████████████████████████| 1.1MB 46.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=859430887c38f898f41de78a1a067a1ed7110605ccfbea151ebeb8ecfef6fa9b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [36]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

In [0]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].text.values,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
label_train = torch.tensor(df[df.data_type=='train'].label.values)


input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
label_val = torch.tensor(df[df.data_type=='val'].label.values)

In [0]:
dataset_train = TensorDataset(input_ids_train,attention_masks_train,label_train)
dataset_val = TensorDataset(input_ids_val,attention_masks_val,label_val)

# Task 4 : Setup Pretrained model

In [0]:
from transformers import BertForSequenceClassification

In [0]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(labels_dict),
    output_attentions = False,
    output_hidden_states = False
)

# Task 5 : Data Loaders

In [0]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

In [0]:
batch_size = 32

In [0]:
dataloader_train = DataLoader(dataset_train,sampler=RandomSampler(dataset_train),batch_size=batch_size)
dataloader_val = DataLoader(dataset_val,sampler=RandomSampler(dataset_val),batch_size=batch_size)


# Task 6 : Optimizer and Scheduler

In [0]:
from transformers import AdamW,get_linear_schedule_with_warmup

In [0]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [0]:
epochs = 10 

schedular = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train) * epochs
)

# Task 7 : Defining Performance Metrics

In [0]:
import numpy as np
from sklearn.metrics import f1_score

In [0]:
def f1_score_func(preds,labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat,preds_flat,average='weighted')
  



# Task 8 : Train the model

In [0]:
import random

In [0]:
seed = 17
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)